In [3]:
!pip install ultralytics==8.0.106

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 15.3 MB/s eta 0:00:00


In [29]:
import ultralytics

In [30]:
from ultralytics.vit import SAM

model = SAM('sam_b.pt')

/usr/local/lib/python3.10/dist-packages/ultralytics/vit/sam/build.py:103: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


In [6]:
model.predict('/content/images/dog2.jpg')

Ultralytics YOLOv8.0.106 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

image 1/1 /content/images/dog2.jpg: 200x3 6345.3ms
Speed: 0.0ms preprocess, 6345.3ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/segment/predict


[ultralytics.yolo.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 keys: ['masks']
 masks: ultralytics.yolo.engine.results.Masks object
 names: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23}
 orig_img: array([[[166, 183, 179],
         [124, 139, 135],
         [126, 140, 136],
         ...,
         [255, 254, 255],
         [255, 254, 255],
         [255, 254, 255]],
 
        [[180, 195, 191],
         [136, 150, 146],
         [152, 163, 160],
         ...,
         [255, 254, 255],
         [255, 254, 255],
         [255, 254, 255]],
 
        [[154, 166, 160],
         [133, 143, 137],
         [224, 230, 225],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[222, 207, 204],
         [222, 207, 204],
         [221, 206, 203],
         ...,
         [227, 2

In [31]:
from ultralytics.yolo.data.annotator import auto_annotate

In [32]:
auto_annotate(data="images", det_model="yolov8x.pt", sam_model='sam_b.pt')

Ultralytics YOLOv8.0.106 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

image 1/4 /content/images/car.jpg: 384x640 1 truck, 57.7ms
image 2/4 /content/images/cat.jpg: 480x640 1 cat, 43.0ms
image 3/4 /content/images/dog1.jpg: 640x512 1 dog, 45.6ms
image 4/4 /content/images/dog2.jpg: 448x640 1 dog, 43.1ms
Speed: 2.7ms preprocess, 47.3ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)


In [9]:
import cv2
import os
import numpy as np  # Add this for handling polygons

def draw_yolo_labels(image_path, label_path, output_path, class_names=None):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Read the label file
    with open(label_path, "r") as file:
        labels = file.readlines()

    for label in labels:
        label_parts = label.strip().split()
        class_id = int(label_parts[0])  # Extract the class ID

        # Check if it is a polygon (more than 4 values after class_id)
        if len(label_parts) > 5:
            # Extract polygon points
            points = list(map(float, label_parts[1:]))
            polygon = []
            for i in range(0, len(points), 2):
                x = int(points[i] * width)
                y = int(points[i + 1] * height)
                polygon.append((x, y))

            # Draw the polygon
            polygon = np.array(polygon, np.int32)
            polygon = polygon.reshape((-1, 1, 2))
            cv2.polylines(image, [polygon], isClosed=True, color=(0, 255, 0), thickness=2)

        else:
            # Extract bounding box coordinates
            x_center, y_center, box_width, box_height = map(float, label_parts[1:])
            x1 = int((x_center - box_width / 2) * width)
            y1 = int((y_center - box_height / 2) * height)
            x2 = int((x_center + box_width / 2) * width)
            y2 = int((y_center + box_height / 2) * height)

            # Draw the bounding box
            cv2.rectangle(image, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)

        # Add class name if provided
        if class_names:
            class_name = class_names[class_id] if class_id < len(class_names) else str(class_id)
            if len(label_parts) > 5:  # Polygon case
                text_x, text_y = polygon[0][0][0], polygon[0][0][1]
            else:  # Bounding box case
                text_x, text_y = x1, y1
            cv2.putText(image, class_name, (text_x, text_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the image with annotations
    cv2.imwrite(output_path, image)


# List of images and labels
images = ["/content/images/dog1.jpg", "/content/images/dog2.jpg"]
labels = ["/content/labels/dog1.txt", "/content/labels/dog2.txt"]
output_dir = "/content/annotated_images/"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process each image and label
for image_path, label_path in zip(images, labels):
    output_path = os.path.join(output_dir, os.path.basename(image_path))  # Save with the same name as input image
    draw_yolo_labels(image_path, label_path, output_path, class_names='dog')  # Add class_names if available


In [18]:
import cv2
import os
import numpy as np  # For handling polygons
from ultralytics import YOLO  # YOLO library for extracting class names

def draw_yolo_labels(image_path, label_path, output_path, class_names=None):
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Read the label file
    with open(label_path, "r") as file:
        labels = file.readlines()

    for label in labels:
        label_parts = label.strip().split()
        class_id = int(label_parts[0])  # Extract the class ID

        # Check if it is a polygon (more than 4 values after class_id)
        if len(label_parts) > 5:
            # Extract polygon points
            points = list(map(float, label_parts[1:]))
            polygon = []
            for i in range(0, len(points), 2):
                x = int(points[i] * width)
                y = int(points[i + 1] * height)
                polygon.append((x, y))

            # Draw the polygon
            polygon = np.array(polygon, np.int32)
            polygon = polygon.reshape((-1, 1, 2))
            cv2.polylines(image, [polygon], isClosed=True, color=(0, 255, 0), thickness=2)

        else:
            # Extract bounding box coordinates
            x_center, y_center, box_width, box_height = map(float, label_parts[1:])
            x1 = int((x_center - box_width / 2) * width)
            y1 = int((y_center - box_height / 2) * height)
            x2 = int((x_center + box_width / 2) * width)
            y2 = int((y_center + box_height / 2) * height)

            # Draw the bounding box
            cv2.rectangle(image, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)

        # Add class name if provided
        if class_names:
            class_name = class_names[class_id] if class_id < len(class_names) else str(class_id)
            if len(label_parts) > 5:  # Polygon case
                text_x, text_y = polygon[0][0][0], polygon[0][0][1]
            else:  # Bounding box case
                text_x, text_y = x1, y1
            # Change text color to blue (BGR: 255, 0, 0)
            cv2.putText(image, class_name, (text_x, text_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Save the image with annotations
    cv2.imwrite(output_path, image)


# Load the YOLO model
yolo_model = YOLO("yolov8n.pt")  # Load YOLOv8 model (use your desired model here)
class_names = yolo_model.names  # Extract class names from the YOLO model

# List of images and labels
images = ["/content/images/dog1.jpg", "/content/images/dog2.jpg","/content/images/cat.jpg"]
labels = ["/content/labels/dog1.txt", "/content/labels/dog2.txt","/content/labels/cat.txt"]
output_dir = "/content/annotated_images/"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process each image and label
for image_path, label_path in zip(images, labels):
    output_path = os.path.join(output_dir, os.path.basename(image_path))  # Save with the same name as input image
    draw_yolo_labels(image_path, label_path, output_path, class_names=class_names)


In [19]:
print(class_names)  # Check if "cat" exists in the loaded YOLO model


{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [20]:
from ultralytics import YOLO

# Load the YOLOv8 model
yolo_model = YOLO("yolov8n.pt")  # Use the appropriate model file (e.g., yolov8s.pt, yolov8m.pt)

# Path to the image you want to test
image_path = "/content/images/cat.jpg"

# Run prediction
results = yolo_model.predict(source=image_path, conf=0.25, save=True, save_txt=True)  # Adjust confidence as needed

# Print the results
print("Prediction Results:")
for result in results:
    print(result.boxes)  # Prints the bounding boxes and class IDs

# Extract detected classes
print("\nDetected Classes:")
for result in results:
    for box in result.boxes.data.tolist():
        class_id = int(box[5])  # Assuming class ID is in index 5
        confidence = box[4]  # Assuming confidence is in index 4
        print(f"Class ID: {class_id}, Confidence: {confidence}")



image 1/1 /content/images/cat.jpg: 480x640 1 cat, 61.2ms
Speed: 2.9ms preprocess, 61.2ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels
WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


Prediction Results:
ultralytics.yolo.engine.results.Boxes object with attributes:

boxes: tensor([[ 60.1292,   9.0095, 380.0529, 375.0000,   0.8288,  15.0000]], device='cuda:0')
cls: tensor([15.], device='cuda:0')
conf: tensor([0.8288], device='cuda:0')
data: tensor([[ 60.1292,   9.0095, 380.0529, 375.0000,   0.8288,  15.0000]], device='cuda:0')
id: None
is_track: False
orig_shape: tensor([375, 500], device='cuda:0')
shape: torch.Size([1, 6])
xywh: tensor([[220.0910, 192.0047, 319.9238, 365.9905]], device='cuda:0')
xywhn: tensor([[0.4402, 0.5120, 0.6398, 0.9760]], device='cuda:0')
xyxy: tensor([[ 60.1292,   9.0095, 380.0529, 375.0000]], device='cuda:0')
xyxyn: tensor([[0.1203, 0.0240, 0.7601, 1.0000]], device='cuda:0')

Detected Classes:
Class ID: 15, Confidence: 0.8288013935089111


In [35]:
import cv2
import os
import numpy as np
from ultralytics import YOLO

def draw_yolo_labels(image_path, label_path, output_path, model_path="yolov8n.pt"):
    """
    Annotates an image using the provided YOLO labels.

    Args:
        image_path (str): Path to the input image.
        label_path (str): Path to the YOLO label file.
        output_path (str): Path to save the annotated image.
        model_path (str): Path to the YOLO model (unused in this version).
    """
    # Load the image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Get class names from the YOLO model (this part is optional if you just need the labels)
    yolo_model = YOLO(model_path)
    class_names = yolo_model.names

    # 1. Draw from label file if it exists
    if os.path.exists(label_path):
        with open(label_path, "r") as file:
            labels = file.readlines()
        for label in labels:
            label_parts = label.strip().split()
            class_id = int(label_parts[0])  # Class ID

            if len(label_parts) == 5:  # Bounding box format
                x_center, y_center, box_width, box_height = map(float, label_parts[1:])

                # Convert YOLO format to pixel coordinates
                x1 = int((x_center - box_width / 2) * width)
                y1 = int((y_center - box_height / 2) * height)
                x2 = int((x_center + box_width / 2) * width)
                y2 = int((y_center + box_height / 2) * height)

                # Draw bounding box
                cv2.rectangle(image, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)  # Blue for labels

                # Add class name
                class_name = class_names[class_id] if class_id < len(class_names) else str(class_id)
                cv2.putText(image, f"{class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            elif len(label_parts) > 5:  # Polygon format
                points = list(map(float, label_parts[1:]))
                polygon = []
                for i in range(0, len(points), 2):
                    x = int(points[i] * width)
                    y = int(points[i + 1] * height)
                    polygon.append((x, y))

                # Draw the polygon
                polygon = np.array(polygon, np.int32).reshape((-1, 1, 2))
                cv2.polylines(image, [polygon], isClosed=True, color=(255, 0, 0), thickness=2)

                # Add class name
                class_name = class_names[class_id] if class_id < len(class_names) else str(class_id)
                text_x, text_y = polygon[0][0]
                cv2.putText(image, f"{class_name}", (text_x, text_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Save the annotated image
    cv2.imwrite(output_path, image)
    print(f"Annotated image saved at: {output_path}")


# Directories containing images and labels
images_dir = "/content/images"
labels_dir = "/content/labels"
output_dir = "/content/outputs"  # Path to save the annotated images
model_path = "yolov8n.pt"  # Path to your YOLOv8 model (unused in this version)

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process each image and its corresponding label file
for image_filename in os.listdir(images_dir):
    # Match the image with its label (assumes the label file has the same name as the image file)
    image_path = os.path.join(images_dir, image_filename)
    label_filename = os.path.splitext(image_filename)[0] + ".txt"
    label_path = os.path.join(labels_dir, label_filename)

    # Ensure label file exists
    if os.path.exists(label_path):
        output_filename = os.path.splitext(image_filename)[0] + "_annotated.jpg"
        output_path = os.path.join(output_dir, output_filename)

        # Run the annotation function
        draw_yolo_labels(image_path, label_path, output_path, model_path=model_path)
    else:
        print(f"Label file not found for {image_filename}. Skipping.")


Annotated image saved at: /content/outputs/car_annotated.jpg
Annotated image saved at: /content/outputs/cat_annotated.jpg
Annotated image saved at: /content/outputs/dog2_annotated.jpg
Annotated image saved at: /content/outputs/dog1_annotated.jpg
